In [1]:
import os
from load_dataset import load_corpus, load_queries, load_retrieval
from crossencoder_bm25 import CustomCrossEncoder
import numpy as np

/tmp/ipykernel_130013/1964598435.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
data_folder = "./data/msmarco-passage/"
corpus = load_corpus(os.path.join(data_folder, "collection.tsv"))
queries = load_queries(os.path.join(data_folder, "queries.dev.small.tsv"))
retrieval_samples = load_retrieval(
        os.path.join(data_folder, "msmarco.bm25.dev.small.tsv.gz"), corpus, queries
    )



8.84Mit [00:11, 776kit/s]
809kit [00:00, 1.71Mit/s]
543kit [00:00, 1.29Mit/s]


In [3]:
# First, we define the transformer model we want to fine-tune
model_name = "output/training_ms-marco_cross-encoder-microsoft-MiniLM-L12-H384-uncased-2023-05-20_12-13-19-latest"

# We set num_labels=1, which predicts a continous score between 0 and 1
model = CustomCrossEncoder(model_name, num_labels=1, max_length=512)


In [ ]:
# Compute the similarity scores for these combinations
similarity_scores = model.predict(retrieval_samples)

In [ ]:
# Sort the scores in decreasing order
sim_scores_argsort = reversed(np.argsort(similarity_scores))